In [1]:
import os, json
from datetime import datetime, timedelta, timezone
import pandas as pd

In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# read files

In [3]:
vault = 'DynYieldB'
#vault = 'DynLst'
dirname = f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}"
pools = []
for filename in os.listdir(dirname):
    fullname = os.path.join(os.sep, dirname, filename)
    data = pd.read_csv(fullname, parse_dates=['date']).set_index('date')['haircut_apy']
    pools.append(data.rename(filename.replace('.csv','')))
history = pd.concat(pools, axis=1)

In [4]:
metadata = pd.read_csv(f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}_pool_metadata.csv")
pool_map = metadata[['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
pool_map

{'9f3a6015-5045-4471-ba65-ad3dc7c38269': 'venus_USDT',
 '282baaa9-11ef-4e72-8ade-efe0c43c07cc': 'pancakeswap-amm-v3_USDT-USDC',
 'a2e7bb87-b339-44d9-91d7-87a04552afe9': 'stargate_USDT',
 '9f0f4cbd-5a26-4eb8-b720-3691b42b0e56': 'pancakeswap-amm-v3_USDT-BUSD',
 '89eba1e5-1b1b-47b6-958b-38138a04c244': 'venus_USDC',
 '0e66dcfe-c01e-4c3b-81f9-99e0a6ab0903': 'pancakeswap-amm-v3_USDC-BUSD',
 'b2e9d5fe-19ca-456b-8cd5-c296d635136f': 'pancakeswap-amm-v3_USDT-USDC',
 '72a0cd12-822f-4b71-a063-9887fbfff463': 'stargate_BUSD',
 'b6fa8f40-7e6b-414e-850d-807750249031': 'pancakeswap-amm_USDT-BUSD',
 '18abc39e-152c-46ce-933d-b7db5609a78a': 'thena-v1_USDT-BUSD'}

# compute historical pool rank

In [5]:
max_rank = 10
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 90)
df = history.fillna(method='ffill')
df = df[(df.index >= start)&(df.index <= end)]
best_only = df[df.rank(axis=1, ascending=False)<max_rank].dropna(how='all', axis=1)
#best_only = best_only.rename(columns=pool_map)
best_only.iplot()

In [6]:
# with open(os.path.join(os.sep, dirname, f'EverTop{max_rank}.json'), 'w') as fp:
#     json.dump(list(best_only.columns), fp=fp)

In [7]:
short_listed_protocols = [
                'lido',
            'frax-ether',
            'rocket-pool',
            'aave-v3',
            'morpho-aavev3',
            'balancer-v2',
            'curve-finance',
            'convex-finance',
            'aura',
            'compound',
            'morpho-compound'
            'compound-v3',
            'morpho-aave',
            'convex-finance',
            'uniswap-v3',
            'uniswap-v2',
            'aave-v2',
            'aave-v3',
            'curve-dex',
            'spark',
            'makerdao',
            'aura',
            'balancer',
'stargate',
                                                 'venus',
                                                 'pancakeswap-amm-v3',
                                                 'wing-finance',
                                                 'thena-v1',
                                                 'pancakeswap-amm']
for it in best_only.columns:
    descr = pool_map[it] if it in pool_map else []
    if any(protocol in descr for protocol in short_listed_protocols):
        print(f'- \"{it}\" # {descr}')

- "0e66dcfe-c01e-4c3b-81f9-99e0a6ab0903" # pancakeswap-amm-v3_USDC-BUSD
- "282baaa9-11ef-4e72-8ade-efe0c43c07cc" # pancakeswap-amm-v3_USDT-USDC
- "89eba1e5-1b1b-47b6-958b-38138a04c244" # venus_USDC
- "9f0f4cbd-5a26-4eb8-b720-3691b42b0e56" # pancakeswap-amm-v3_USDT-BUSD
- "9f3a6015-5045-4471-ba65-ad3dc7c38269" # venus_USDT
- "b2e9d5fe-19ca-456b-8cd5-c296d635136f" # pancakeswap-amm-v3_USDT-USDC
